### text embeddings
pretrained embeddings 모델(BERT 등) 에서 chunk 의 숫자화(벡터화)로 문장간의 유사성 비교 가능하게 한다.

즉 비정형 데이터의 정형화

- API 모델 (좋은 퀄리티, 보안 우려)
    - openai : `text-embedding-ada-002`
    - cohera : `embed-multilingual-v2.0`
    - amazon : `titan-embed-text-v1`
- huggingface (직접 수행해야할 것이 많다, **보안!**) : 리더보드로 순위를 보고 선택 가능
    - `bge-large-en-v1.5`
    - `multilingual-e5-large`
    - `instructor-xl`
    - 한국어 가능 : 
        - `**ko-sbert-nli**`
        - `**koSimCSE-roberta-multitask 등**`

In [1]:
from langchain_openai import OpenAIEmbeddings
import json

with open("api_keys.json") as f:
    OPENAI_API_KEY = json.load(f)["openai"]

embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [2]:
docs =["안녕 하세요",
       "저는",
       "ai 개발 취업 준비를 하고 있습니다",
       "반갑습니다.",
       "im from south korea",
       "thank you."]

embeddings = embeddings_model.embed_documents(docs)

In [3]:
print(f"전체 임베딩 포인트 수:{len(embeddings)}, 임베딩 차원수 : {len(embeddings[1])}")

전체 임베딩 포인트 수:6, 임베딩 차원수 : 1536


In [4]:
embeddings_model

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7fee609d27d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7fee609bd490>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [5]:
queary_q = embeddings_model.embed_query("화자는 어떤 직업을 고려하고 있나요?")
queary_a = embeddings_model.embed_query("ai 모델을 개발하는 직업을 고려하고 있습니다.")

In [6]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

In [7]:
print(cosine_similarity(queary_q, queary_a))
for d,e in zip(docs,embeddings):
    print(f"유사도 : {cosine_similarity(queary_q, e):0.3f} with 질문 and '{d}'")

0.8728972445836978
유사도 : 0.808 with 질문 and '안녕 하세요'
유사도 : 0.805 with 질문 and '저는'
유사도 : 0.837 with 질문 and 'ai 개발 취업 준비를 하고 있습니다'
유사도 : 0.803 with 질문 and '반갑습니다.'
유사도 : 0.773 with 질문 and 'im from south korea'
유사도 : 0.695 with 질문 and 'thank you.'


### huggingface 임베딩 모델 활용

In [8]:
# !pip install sentence_transformers
# !pip install einops

In [22]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}

hf_embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs)

In [23]:
embeddings = hf_embedding_model.embed_documents(docs)

In [24]:
queary_q = hf_embedding_model.embed_query("화자는 어떤 직업을 고려하고 있나요?")
queary_a = hf_embedding_model.embed_query("ai 모델을 개발하는 직업을 고려하고 있습니다.")

In [25]:
print(cosine_similarity(queary_q, queary_a))
for d,e in zip(docs,embeddings):
    print(f"유사도 : {cosine_similarity(queary_q, e):0.3f} with 질문 and '{d}'")

0.8531398029769677
유사도 : 0.865 with 질문 and '안녕 하세요'
유사도 : 0.840 with 질문 and '저는'
유사도 : 0.829 with 질문 and 'ai 개발 취업 준비를 하고 있습니다'
유사도 : 0.886 with 질문 and '반갑습니다.'
유사도 : 0.767 with 질문 and 'im from south korea'
유사도 : 0.716 with 질문 and 'thank you.'


- transformers 라이브러리 이용

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn.functional as F

model_name = "BM-K/KoSimCSE-roberta-multitask"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModel.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def encode_sentences(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    embeddings = model(**inputs, return_dict=False)[0]
    # Mean pooling 적용
    embeddings = embeddings.mean(dim=1)
    # 임베딩 정규화
    embeddings = embeddings / embeddings.norm(dim=1)[:, None]
    return embeddings

def cosine_similarity_torch(a, b):
    similarity = F.cosine_similarity(a, b) # 코사인 유사도 함수 이용
    similarity = similarity.cpu().item()
    return similarity


In [74]:
queary_q = encode_sentences("화자는 어떤 직업을 고려하고 있나요?")
queary_a = encode_sentences("ai 모델을 개발하는 직업을 고려하고 있습니다.")

In [75]:
docs

['안녕 하세요',
 '저는',
 'ai 개발 취업 준비를 하고 있습니다',
 '반갑습니다.',
 'im from south korea',
 'thank you.']

In [76]:
print(cosine_similarity_torch(queary_q, queary_a))

0.5752071738243103


In [77]:
print(cosine_similarity_torch(queary_q, queary_a))
for d,e in zip(docs,embeddings):
    print(f"유사도 : {cosine_similarity_torch(queary_q, e):0.3f} with 질문 and '{d}'")

0.5752071738243103
유사도 : 0.269 with 질문 and '안녕 하세요'
유사도 : 0.465 with 질문 and '저는'
유사도 : 0.501 with 질문 and 'ai 개발 취업 준비를 하고 있습니다'
유사도 : 0.286 with 질문 and '반갑습니다.'
유사도 : 0.156 with 질문 and 'im from south korea'
유사도 : 0.226 with 질문 and 'thank you.'
